Tests performance of various nearest neighbors classifiers using similartiy scores derived from pretrained arcface embeddings.

End goal is to have reasonably good results when searching by an identity. Also need a way to reliably identify probes that are not in the gallery.

One solution is to use a hybrid of k nearest neighbors and radius neighbors. Remove all neighbors with too great a distance, then have voting by k-nearest. If no neighbors are present, label probe as outlier (ie, not in gallery)

In the end, radius neighbors with weighting by distance ends up giving better performance anyway. Ends up getting 100% accuracy on current data set.

#### To do

Need to return with a larger number of classes to better test performance. Can also add more non-gallery probes by pulling from megaface.

Also need to optimize on precision and recall once there's a bigger dataset.

In [1]:
import torch
from torch.utils.data import DataLoader,  SequentialSampler
from torchvision import datasets, transforms
import numpy as np
import os

In [2]:
data_dir = 'data/zack_erin_other'

batch_size = 16
epochs = 15
workers = 0 if os.name == 'nt' else 8

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [4]:
from data_gen import data_transforms

In [5]:
#create dataset and data loaders from cropped images output from MTCNN

# trans = transforms.Compose([
#     np.float32,
#     transforms.ToTensor(),
#     fixed_image_standardization
# ])

trans = data_transforms['val']

#Training set can be much smaller because we aren't actually training, just creating a "template" in the gallery
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))

classes = dataset.classes
print(classes)

#no need to randomize. there will be only one epoch. Basically don't need the dataloader except for batch control
embed_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SequentialSampler(dataset)
)

In [6]:
checkpoint = 'BEST_checkpoint_r101.tar'
checkpoint = torch.load(checkpoint)
model = checkpoint['model'].module
model = model.to(device)
model.eval()

/home/zack/miniconda3/envs/pytorch/lib/python3.7/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.parallel.data_parallel.DataParallel' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/zack/miniconda3/envs/pytorch/lib/python3.7/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/zack/miniconda3/envs/pytorch/lib/python3.7/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has changed. you can

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (prelu): PReLU(num_parameters=1)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): IRBlock(
      (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (prelu): PReLU(num_parameters=1)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (se): SEBlock(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=64, out_features=4, bias=Tru

In [7]:
labels = []
embeddings = []
imgs = []
with torch.no_grad():
    for xb, yb in embed_loader:

        xb = xb.to(device)
        b_embeddings = model(xb)
        b_embeddings = b_embeddings.to('cpu').numpy()
        labels.extend(yb.numpy())
        embeddings.extend(b_embeddings)
        imgs.extend(xb)


In [8]:
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.base import BaseEstimator

train_embeddings, val_embeddings, train_labels, val_labels, train_imgs, val_imgs = map(np.array, train_test_split(embeddings, labels, imgs,test_size=.5))

In [9]:
def angular_distance(feature0,feature1):
    x0 = feature0 / np.linalg.norm(feature0)
    x1 = feature1 / np.linalg.norm(feature1)
    cosine = np.dot(x0,x1)
    cosine = np.clip(cosine, -1.0, 1.0)
    theta = np.arccos(cosine)
    theta = theta * 180 / np.pi

    return theta

# def cosine_similarity(x1, x2):
#     return np.dot(x1, x2) / (np.linalg.norm(x1) * np.linalg.norm(x2))

class ThresholdKNN(BaseEstimator):
    def __init__(self,  n_neighbors, threshold, metric, outlier_value):
        self.n_neighbors = n_neighbors
        self.threshold = threshold
        self.metric = metric
        self.outlier_value = outlier_value

    def fit(self,X,y):
        self.train_embeddings = np.array(X)
        self.train_labels = np.array(y)

    def predict(self, X):
        sims = []
        preds = []
        for x in X:
            sim = []
            for train_embedding in self.train_embeddings:
                sim.append(self.metric(x, train_embedding))
            sims.append(np.array(sim))

        for sim in sims:
            threshold_mask = sim < self.threshold
            sorted_index = np.argsort(sim)
            k_neighbors_index = sorted_index[threshold_mask[sorted_index]][:self.n_neighbors]
            if len(k_neighbors_index) >0:
                k_neighbors = self.train_labels[k_neighbors_index]
                votes = np.unique(k_neighbors, return_counts=True)
                highest_votes = votes[0][np.argmax(votes[1])]
                preds.append(highest_votes)
            else:
                preds.append(self.outlier_value)

        return np.array(preds)

    def score(self, X, y):
        accuracy = np.sum(np.equal(self.predict(X), y))/len(y)
        return accuracy

#     def get_params(self, deep=True):
#         return dict(n_neighbors=self.n_neighbors,threshold=self.threshold, distance_fn=self.distance_fn, outlier_value=self.outlier_value)
#
#     def set_params(self,):
#

In [10]:
# knn = ThresholdKNN(k=5, threshold=75, distance_fn=angular_distance, outlier_value =-1)
# knn.fit(train_embeddings,train_labels)
# val_preds = knn.predict(val_embeddings)
# accuracy_score(val_labels, val_preds)

In [11]:
#knn.score(val_embeddings,val_labels)

In [12]:
# def find_best_threshold(val_embedding, val_labels, train_embeddings, train_labels, thresholds, majority_req):
#     accuracy = []
#     for threshold in thresholds:
#         val_preds = []
#         for val_embedding in val_embeddings:
#             val_preds.append(threshold_nearest_neighbor(val_embedding, train_embeddings, train_labels, threshold, majority_req))
#         accuracy.append(accuracy_score(val_labels,val_preds))
#     return [x for x in zip(thresholds,accuracy)]

In [13]:
# thresholds = thresholds = np.arange(0, 200,2)
# find_best_threshold(val_embedding, val_labels, train_embeddings, train_labels, thresholds, False)


In [14]:
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier
from scipy.spatial.distance import cosine as cosine_distance

cv = StratifiedKFold(5, shuffle=True, random_state=0)
knn = KNeighborsClassifier(n_neighbors=3, metric=angular_distance)
threshold_knn = ThresholdKNN(n_neighbors=5, threshold=75, metric=angular_distance, outlier_value =1)
radius_knn = RadiusNeighborsClassifier(radius=70, metric=angular_distance,outlier_label=1)



In [15]:
 from sklearn.model_selection import GridSearchCV
#
# knn_params = dict(n_neighbors=range(1,16), metric=[angular_distance,cosine_distance])
# knn_grid = GridSearchCV(knn, param_grid=knn_params)
# knn_grid.fit(embeddings,labels)

In [16]:
# print(knn_grid.best_params_)
# knn_grid.best_estimator_.score(val_embeddings,val_labels)

In [17]:
threshold_knn_params = dict(n_neighbors=range(3,10), threshold=range(65,86), metric=[angular_distance])
threshold_knn_grid = GridSearchCV(threshold_knn, param_grid=threshold_knn_params, n_jobs=8)
threshold_knn_grid.fit(embeddings,labels)

GridSearchCV(estimator=ThresholdKNN(metric=<function angular_distance at 0x7f6fa3a6a440>,
                                    n_neighbors=5, outlier_value=1,
                                    threshold=75),
             param_grid={'metric': [<function angular_distance at 0x7f6fa3a6a440>],
                         'n_neighbors': range(1, 16),
                         'threshold': range(65, 86)})

In [18]:
print(threshold_knn_grid.best_params_)
threshold_knn_grid.best_estimator_.score(val_embeddings,val_labels)


{'metric': <function angular_distance at 0x7f6fa3a6a440>, 'n_neighbors': 3, 'threshold': 70}


0.9836065573770492

In [22]:
radius_knn_params = dict(radius=range(65,86,2), metric=[angular_distance], weights=['uniform','distance'])
radius_knn_grid = GridSearchCV(radius_knn, param_grid=radius_knn_params, n_jobs=8)
radius_knn_grid.fit(embeddings,labels)

GridSearchCV(estimator=RadiusNeighborsClassifier(metric=<function angular_distance at 0x7f6fa3a6a440>,
                                                 outlier_label=1, radius=70),
             n_jobs=8,
             param_grid={'metric': [<function angular_distance at 0x7f6fa3a6a440>],
                         'radius': range(65, 86, 2),
                         'weights': ['uniform', 'distance']})

In [23]:
print(radius_knn_grid.best_params_)
radius_knn_grid.best_estimator_.score(val_embeddings,val_labels)


{'metric': <function angular_distance at 0x7f6fa3a6a440>, 'radius': 73, 'weights': 'distance'}


1.0